In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [8]:
# URL of the Falcon 9 and Falcon Heavy launches Wikipedia page
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"


In [10]:

# Requesting the page
response = requests.get(static_url)

# Creating the BeautifulSoup object
soup = BeautifulSoup(response.text, 'html.parser')

# Find all tables on the page
tables = soup.find_all('table')

# List to hold all column names from each table
all_columns = []

# Loop through each table and extract column names (headers)
for i, table in enumerate(tables, 1):
    headers = table.find_all('th')  # Find all <th> tags (table headers)
    column_names = [header.get_text(strip=True) for header in headers]  # Extract text from headers
    if column_names:  # Only append if there are headers
        all_columns.append((f"Table {i}", column_names))  # Adding table number to column names

# Convert to a DataFrame for better presentation
df = pd.DataFrame(all_columns, columns=['Table', 'Column Names'])

# Display the result in table format
print(df)


       Table                                       Column Names
0    Table 3  [Flight No., Date andtime (UTC), Version,Boost...
1    Table 4  [Flight No., Date andtime (UTC), Version,Boost...
2    Table 5  [Flight No., Date andtime (UTC), Version,Boost...
3    Table 6  [Flight No., Date andtime (UTC), Version,Boost...
4    Table 7  [Flight No., Date andtime (UTC), Version,Boost...
5    Table 8  [Flight No., Date andtime (UTC), Version,Boost...
6    Table 9  [Flight No., Date andtime (UTC), Version,Boost...
7   Table 10  [Flight No., Date andtime (UTC), Version,Boost...
8   Table 11  [FlightNo., Date andtime (UTC), Version,Booste...
9   Table 12  [Date and time (UTC), Version,Booster[b], Laun...
10  Table 13  [Date and time (UTC), Version,Booster[b], Laun...
11  Table 14  [Date and time (UTC), Version,Booster[b], Laun...
12  Table 15  [Date and time (UTC), Version,Booster[b], Laun...
13  Table 16  [vteSpaceX missions and payloads, Launch vehic...
14  Table 17  [Demonstrations, ISSlogist

In [12]:

from bs4 import BeautifulSoup
import requests
import pandas as pd

# Define the extract_column_from_header function
def extract_column_from_header(header):
    # Apply any specific logic to extract the column name (e.g., strip whitespace, format)
    column_name = header.get_text(strip=True)
    return column_name if column_name else None

# URL of the Falcon 9 and Falcon Heavy launches Wikipedia page
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Requesting the page
response = requests.get(static_url)

# Creating the BeautifulSoup object
soup = BeautifulSoup(response.text, 'html.parser')

# Find all tables on the page
tables = soup.find_all('table')

# Skip the first two tables, start from the third table (index 2)
target_table = tables[2]  # Third table (index 2)

# Extract column names
column_names = []
headers = target_table.find_all('th')  # Find all <th> elements in the third table

# Iterate through each <th> element and apply extract_column_from_header() to get column names
for header in headers:
    name = extract_column_from_header(header)
    if name and len(name) > 0:  # Only add non-empty column names
        column_names.append(name)

# Create a dictionary from column names
launch_dict = dict.fromkeys(column_names)

# Remove an irrelevant column
if 'Date and time ( )' in launch_dict:
    del launch_dict['Date and time ( )']

# Initialize the dictionary with each value as an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []

# Add some new columns
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

# Extract rows of data
rows = target_table.find_all('tr')

# Loop through each row (starting from the second row to skip the header)
for row in rows[1:]:
    cells = row.find_all('td')  # Find all data cells in the row
    if len(cells) == len(column_names):  # Ensure the row has the same number of columns as headers
        # Extract the data for each column
        launch_dict['Flight No.'].append(cells[0].get_text(strip=True))
        launch_dict['Launch site'].append(cells[1].get_text(strip=True))
        launch_dict['Payload'].append(cells[2].get_text(strip=True))
        launch_dict['Payload mass'].append(cells[3].get_text(strip=True))
        launch_dict['Orbit'].append(cells[4].get_text(strip=True))
        launch_dict['Customer'].append(cells[5].get_text(strip=True))
        launch_dict['Launch outcome'].append(cells[6].get_text(strip=True))
        # Additional columns
        launch_dict['Version Booster'].append(cells[7].get_text(strip=True) if len(cells) > 7 else '')
        launch_dict['Booster landing'].append(cells[8].get_text(strip=True) if len(cells) > 8 else '')
        launch_dict['Date'].append(cells[9].get_text(strip=True) if len(cells) > 9 else '')
        launch_dict['Time'].append(cells[10].get_text(strip=True) if len(cells) > 10 else '')

# Convert the dictionary into a Pandas DataFrame
launch_df = pd.DataFrame(launch_dict)

# Display the DataFrame
print(launch_df)


Empty DataFrame
Columns: [Flight No., Date andtime (UTC), Version,Booster[b], Launch site, Payload[c], Payload mass, Orbit, Customer, Launchoutcome, Boosterlanding, 1, 2, 3, 4, 5, 6, 7, Payload, Launch outcome, Version Booster, Booster landing, Date, Time]
Index: []

[0 rows x 23 columns]


In [14]:
from bs4 import BeautifulSoup
import requests

# Define helper functions for extracting and cleaning data
def date_time(cell):
    # Assuming the first cell contains date and time as a string like "2023-01-01 15:30"
    # Adjust this function as per actual format if needed
    date_time_str = cell.get_text(strip=True)
    return date_time_str.split() if date_time_str else ['', '']

def booster_version(cell):
    # Assuming the second column contains the booster version; we need to clean or extract it
    return cell.get_text(strip=True)

def get_mass(cell):
    # Extract payload mass (if any) from the given cell
    return cell.get_text(strip=True) or 'N/A'

def landing_status(cell):
    # Extract booster landing status (e.g., "Yes" or "No")
    return cell.get_text(strip=True) or 'N/A'

# URL of the Falcon 9 and Falcon Heavy launches Wikipedia page
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Requesting the page
response = requests.get(static_url)

# Creating the BeautifulSoup object
soup = BeautifulSoup(response.text, 'html.parser')

# Initialize an empty dictionary to store the launch data
launch_dict = {
    'Flight No.': [],
    'Date': [],
    'Time': [],
    'Version Booster': [],
    'Launch Site': [],
    'Payload': [],
    'Payload mass': [],
    'Orbit': [],
    'Customer': [],
    'Launch outcome': [],
    'Booster landing': []
}

# Initialize a counter for the number of extracted rows
extracted_row = 0

# Extract each table with the class 'wikitable plainrowheaders collapsible'
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    for rows in table.find_all("tr"):
        # Check if the row has a header (indicating a launch)
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False
        
        # Extract table data from the row if it's a valid launch row (identified by flight number)
        row = rows.find_all('td')
        if flag:
            extracted_row += 1
            
            # Flight Number
            launch_dict['Flight No.'].append(flight_number)
            
            # Date and Time extraction
            date_time_list = date_time(row[0])
            launch_dict['Date'].append(date_time_list[0].strip(','))
            launch_dict['Time'].append(date_time_list[1])
            
            # Booster Version
            bv = booster_version(row[1])
            if not bv:
                bv = row[1].a.string if row[1].a else ''
            launch_dict['Version Booster'].append(bv)
            
            # Launch Site
            launch_site = row[2].a.string if row[2].a else 'N/A'
            launch_dict['Launch Site'].append(launch_site)
            
            # Payload
            payload = row[3].a.string if row[3].a else 'N/A'
            launch_dict['Payload'].append(payload)
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            
            # Orbit
            orbit = row[5].a.string if row[5].a else 'N/A'
            launch_dict['Orbit'].append(orbit)
            
            # Customer
            customer = row[6].a.string if row[6].a else 'N/A'
            launch_dict['Customer'].append(customer)
            
            # Launch Outcome
            launch_outcome = list(row[7].strings)[0] if row[7].strings else 'N/A'
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Booster Landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

# Convert the dictionary into a pandas DataFrame
import pandas as pd
launch_df = pd.DataFrame(launch_dict)

# Display the DataFrame
print(launch_df)


    Flight No. Date      Time       Version Booster Launch Site  \
0            1    4      June  F9 v1.0[7]B0003.1[8]       CCAFS   
1            2    8  December  F9 v1.0[7]B0004.1[8]       CCAFS   
2            3   22       May  F9 v1.0[7]B0005.1[8]       CCAFS   
3            4    8   October  F9 v1.0[7]B0006.1[8]       CCAFS   
4            5    1     March  F9 v1.0[7]B0007.1[8]       CCAFS   
..         ...  ...       ...                   ...         ...   
116        117    9       May   F9 B5♺B1051.10[657]       CCSFS   
117        118   15       May    F9 B5♺B1058.8[660]         KSC   
118        119   26       May    F9 B5♺B1063.2[665]       CCSFS   
119        120    3      June     F9 B5B1067.1[668]         KSC   
120        121    6      June         F9 B5♺B1061.3       CCSFS   

                                  Payload            Payload mass Orbit  \
0    Dragon Spacecraft Qualification Unit                     N/A   LEO   
1                                  Dragon    

In [15]:
import pandas as pd

# After filling in the launch_dict with the parsed data
df = pd.DataFrame({key: pd.Series(value) for key, value in launch_dict.items()})

# Display the DataFrame
print(df)


    Flight No. Date      Time       Version Booster Launch Site  \
0            1    4      June  F9 v1.0[7]B0003.1[8]       CCAFS   
1            2    8  December  F9 v1.0[7]B0004.1[8]       CCAFS   
2            3   22       May  F9 v1.0[7]B0005.1[8]       CCAFS   
3            4    8   October  F9 v1.0[7]B0006.1[8]       CCAFS   
4            5    1     March  F9 v1.0[7]B0007.1[8]       CCAFS   
..         ...  ...       ...                   ...         ...   
116        117    9       May   F9 B5♺B1051.10[657]       CCSFS   
117        118   15       May    F9 B5♺B1058.8[660]         KSC   
118        119   26       May    F9 B5♺B1063.2[665]       CCSFS   
119        120    3      June     F9 B5B1067.1[668]         KSC   
120        121    6      June         F9 B5♺B1061.3       CCSFS   

                                  Payload            Payload mass Orbit  \
0    Dragon Spacecraft Qualification Unit                     N/A   LEO   
1                                  Dragon    

In [18]:
import json

# Open the .ipynb file and read it
with open('project.ipynb', 'r') as ipynb_file:
    notebook_data = json.load(ipynb_file)

# Save it to a new .json file
with open('project.json', 'w') as json_file:
    json.dump(notebook_data, json_file, indent=4)


In [1]:
import folium

# Step 1: Define the base location for the map (e.g., approximate central location for SpaceX sites)
base_location = [28.5634, -80.5774]  # Latitude and Longitude for Kennedy Space Center

# Step 2: Create the Folium map
spacex_map = folium.Map(location=base_location, zoom_start=6, tiles='OpenStreetMap')

# Step 3: Add SpaceX launch sites as markers
launch_sites = [
    {"name": "CCAFS LC-40", "location": [28.5623, -80.5774], "outcome": "Success"},
    {"name": "KSC LC-39A", "location": [28.573255, -80.646895], "outcome": "Success"},
    {"name": "VAFB SLC-4E", "location": [34.632834, -120.610746], "outcome": "Failure"},
    {"name": "CCAFS SLC-40", "location": [28.563197, -80.576820], "outcome": "Success"}
]

# Step 4: Add markers with different colors based on outcome
for site in launch_sites:
    color = "green" if site["outcome"] == "Success" else "red"
    folium.Marker(
        location=site["location"],
        popup=f"{site['name']} - Outcome: {site['outcome']}",
        icon=folium.Icon(color=color)
    ).add_to(spacex_map)

# Step 5: Save the map to an HTML file
map_path = "spacex_launch_sites_map.html"
spacex_map.save(map_path)

# Output the file path for reference
map_path


'spacex_launch_sites_map.html'

In [9]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Step 1: Load data
# Replace with your dataset or provide a sample dataset here
# Example: SpaceX launch data (mockup for now)
data = {
    'Launch Site': ['CCAFS LC-40', 'KSC LC-39A', 'VAFB SLC-4E', 'CCAFS LC-40'],
    'Payload Mass (kg)': [3000, 4000, 5000, 6000],
    'Orbit': ['LEO', 'GEO', 'LEO', 'GEO'],
    'Outcome': ['Success', 'Success', 'Failure', 'Success']
}
df = pd.DataFrame(data)

# Step 2: Initialize the Dash app
app = dash.Dash(__name__)
app.title = "SpaceX Launch Dashboard"

# Step 3: Define the layout
app.layout = html.Div([
    html.H1("SpaceX Launch Dashboard", style={"textAlign": "center"}),
    
    html.Div([
        html.Label("Select Launch Site:"),
        dcc.Dropdown(
            id="site-dropdown",
            options=[
                {"label": site, "value": site} for site in df['Launch Site'].unique()
            ],
            value="CCAFS LC-40",  # Default value
            placeholder="Select a Launch Site",
        ),
    ], style={"width": "50%", "margin": "auto"}),

    dcc.Graph(id="success-pie-chart"),
    dcc.Graph(id="payload-scatter-plot")
])

# Step 4: Add callback functions
@app.callback(
    [
        Output("success-pie-chart", "figure"),
        Output("payload-scatter-plot", "figure")
    ],
    [Input("site-dropdown", "value")]
)
def update_graphs(selected_site):
    # Filter data based on selected site
    filtered_df = df[df['Launch Site'] == selected_site]

    # Pie Chart: Mission Outcomes
    pie_fig = px.pie(
        filtered_df,
        names="Outcome",
        title=f"Mission Outcomes for {selected_site}"
    )

    # Scatter Plot: Payload vs. Outcome
    scatter_fig = px.scatter(
        filtered_df,
        x="Payload Mass (kg)",
        y="Orbit",
        color="Outcome",
        title=f"Payload vs. Orbit for {selected_site}",
        size="Payload Mass (kg)",
        hover_data=['Outcome']
    )

    return pie_fig, scatter_fig

# Step 5: Run the app
if __name__ == "__main__":
    app.run_server(debug=True)
